# アルゴリズムのパイプラインを改造するためのテンプレート


## はじめに
このNotebookは、04-02.で試してみたパイプラインを改造し、Part3.で学習したAlphaLensを使ってアルファのチューニングを行うためのテンプレートです

### ちょっとしたTips
アルゴリズムのパイプラインを改造する場合、パイプラインのカラムには、アルファのみ入れるようにクセをつけておくと、get_clean_factor_and_forward_returnsの引数処理が汎用的になります。


## 下準備１：（必須）必要なパッケージのインポート
アルファ（ファクター）の構築に必要なパッケージと、Alphalensの利用に必要なパッケージのインポートを行います。
Alphalens 関連のパッケージ以外は、ここまでのNotebookで扱ってきたものです。
必要に応じてパッケージを追加してください。


In [ ]:
# Notebook上でパイプラインを実行する上で必要なパッケージ
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# ファクター 
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data import Fundamentals

# ユニバース
from quantopian.pipeline.filters import QTradableStocksUS

# Alphalens
from alphalens.tears import create_information_tear_sheet
from alphalens.utils import get_clean_factor_and_forward_returns


## 下準備２：（オプション）変数などの指定（オプション）

make_pipeline()の外側で指定している変数を参照しているような場合には、以下のブロックで指定します。


In [ ]:
# Alphalensの分析期間をデフォルト（1D,5D,10D）から変えたいときに利用
periods = (1, 5, 20)

# 変数の指定があれば、ここで便宜的にセットしてしまう
TOTAL_POSITIONS=600


## メイン作業：make_pipline() を貼り付け

make_pipeline()のロジックをコピペします。**以上で作業は終了です**。


In [ ]:
def make_pipeline():
    '''
    アルファの構築を行う
    ここにパイプラインのコードを張り付ける
    '''
    """
    パイプラインを作成して返す関数
    
    パイプラインの構築は、NotebookとAlgorithmの両方で動作させることができるので、Notebook
    で検証したアルファをコピペしてアルゴリズムで使うことが可能。
    【Execise】make_pipeline をNotebookにコピーして動作することを確認しよう。

    Returns
    -------
    pipe : Pipeline
        Represents computation we would like to perform on the assets that make
        it through the pipeline screen.
    """
    
    # ファクターその１：バリューファクター（この時点では生データ）
    value = Fundamentals.ebit.latest / Fundamentals.enterprise_value.latest
    # ファクターその２：クオリティファクター（ROE）（この時点では生データ）
    quality = Fundamentals.roe.latest
    # ファクターその３：センチメントファクター（この時点では生データ）
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )
    
    # ユニバースはQTradableStockUS()を使う
    universe = QTradableStocksUS()
    
    # 生データからwinsorizeによってスコア化
    # バリューファクタースコア
    value_winsorized = value.winsorize(min_percentile=0.05, max_percentile=0.95)
    # クオリティファクタースコア
    quality_winsorized = quality.winsorize(min_percentile=0.05, max_percentile=0.95)
    # センチメントファクタースコア
    sentiment_score_winsorized = sentiment_score.winsorize(min_percentile=0.05, max_percentile=0.95)

    # ３つのファクターを合成して、１つのアルファを構成
    # 【Execise】ファクターの合成方法を変えてバックテストしてみよう！
    combined_factor = (
        value_winsorized.zscore() + 
        quality_winsorized.zscore() + 
        sentiment_score_winsorized.zscore()
    )

    # QTradableStocks構成銘柄の中から、さらに、combined_fatctorが上位のものだけを取得
    longs = combined_factor.top(TOTAL_POSITIONS//2, mask=universe)
    # QTradableStocks構成銘柄の中から、さらに、combined_fatctorが下位のものだけを取得
    shorts = combined_factor.bottom(TOTAL_POSITIONS//2, mask=universe)
    
    # longとshortsの構成銘柄を組み合わせることで、最終的なフィルタを構築
    long_short_screen = (longs | shorts)

    # パイプラインを作成
    pipe = Pipeline(
        columns={
            # 'longs': longs,
            # 'shorts': shorts,
            'combined_factor': combined_factor
        },
        # フィルタリング（longs または shorts に含まれる銘柄でフィルタリング）
        screen=long_short_screen
    )
    
    # 作成したパイプラインを返して終了
    return pipe

# アルファの予測精度分析
コード変更不要

In [ ]:
# Alphalens で分析させる期間（直近から２年分を指定）
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=730)).strftime('%Y-%m-%d')
end_date   = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

# パイプライン実行
pipeline_output = run_pipeline(make_pipeline(), start_date, end_date)

# Alphalenseで必要なデータの準備と実行（この３行を変える必要は多分殆どない）
pricing_data = get_pricing(pipeline_output.index.levels[1], start_date, end_date, fields='open_price')
factor_data = get_clean_factor_and_forward_returns(factor=pipeline_output,
                                                            prices=pricing_data,
                                                            periods=periods)
create_information_tear_sheet(factor_data)

# リターン分析
コード変更不要


In [ ]:
from alphalens.tears import create_returns_tear_sheet
create_returns_tear_sheet(factor_data)

## 最後に
納得のいくパイプラインが出来上がったら、Algorithmの方に張り付けてバックテストを実行する。
